# Temporal model 3


In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, SimpleRNN

In [2]:
import gc
gc.collect()
tf.keras.backend.clear_session()
from numba import cuda 
device = cuda.get_current_device()
device.reset()

## Helper functions

In [2]:
def load_embeddings(subject):
    path = '../embeddings/embeddings_' + subject
    normal_embs = np.load(path + '_normal.npy')
    sleepy_embs = np.load(path + '_sleepy.npy')

    return normal_embs, sleepy_embs

def load_multi_embeddings(subjects):
    normal_dict = {}
    sleepy_dict = {}

    for sub in subjects:
        path = '../embeddings/embeddings_sub' + str(sub)
        normal_frames = np.load(path + '_normal.npy')
        sleepy_frames = np.load(path + '_sleepy.npy')

        normal_dict[str(sub)] = normal_frames
        sleepy_dict[str(sub)] = sleepy_frames

    return normal_dict, sleepy_dict

from helping_functions import *


def get_status_rates(subject, treshhold, segment_length):
    status_rates_sleepy, wrong_frames_sleepy = load_blinks(subject, 'sleepy') 
    status_rates_normal, wrong_frames_normal = load_blinks(subject, 'normal') 
    
    print("Starting segmenting normal condition")
    blink_counts_normal, average_durs_normal = run_analysis(status_rates_normal, wrong_frames_normal, treshhold, segment_length)
    print("Starting segmenting sleepy condition")
    blink_counts_sleepy, average_durs_sleepy = run_analysis(status_rates_sleepy, wrong_frames_sleepy, treshhold, segment_length)
    
    return list(zip(blink_counts_normal, average_durs_normal)), list(zip(blink_counts_sleepy, average_durs_sleepy))

def split_into_segments(frames, gru_segment_length):
    num_segments = len(frames) // gru_segment_length
    frames = frames[:num_segments * gru_segment_length]
    return np.array(np.split(np.array(frames), num_segments))

## First model: GRU deep features

### Model initiation

In [3]:
import gc
gc.collect()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.callbacks import EarlyStopping

class DrowsinessDetector:
    def __init__(self, segment_length, gru_segment_length):
        self.segment_length = segment_length
        self.deep_input_shape = (self.segment_length, 2048)
        self.shallow_input_shape = (gru_segment_length, 2)
        
    def split_frames(self, frames):
        num_segments = len(frames) // self.segment_length
        frames = frames[:num_segments * self.segment_length]
        return np.array(np.split(frames, num_segments))
    
    def create_labels(self, num_segments, label):
        return np.array([label] * num_segments)
    
    def shuffle_data(self, X, y):
        indices = np.arange(len(X))
        np.random.shuffle(indices)
        return X[indices], y[indices]

    def construct_model(self):
        # Define the shallow input layer and GRU
        shallow_input = Input(shape=self.shallow_input_shape)
        shallow_gru = GRU(8)(shallow_input)

        # Define the deep input layer and GRU
        deep_input = Input(shape=self.deep_input_shape)
        deep_gru = GRU(32)(deep_input)

        # Concatenate the outputs of the two GRUs
        merged = concatenate([shallow_gru, deep_gru])

        # Add a dense layer to control how much focus is put on the shallow and deep features
        dense_1 = Dense(16, activation='relu')(merged)

        # Add a dense layer for the final prediction
        dense_2 = Dense(1, activation='sigmoid')(dense_1)

        # Define the model with the two input layers and the concatenated output
        model = Model(inputs=[shallow_input, deep_input], outputs=dense_2)

        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model

    def get_deep_data(self, normal_frames_dict, sleepy_frames_dict, subjects, leave_out_subjects):
            # X_train, y_train = None, None
            X_train, y_train = [], []
            length_list = []
            for subject in subjects:
                if subject in leave_out_subjects:
                    continue
                print('train', subject)
                normal_frames = normal_frames_dict[str(subject)]
                normal_X = self.split_frames(normal_frames)
                normal_y = self.create_labels(len(normal_X), 0)

                sleepy_frames = sleepy_frames_dict[str(subject)]
                sleepy_X = self.split_frames(sleepy_frames)
                sleepy_y = self.create_labels(len(sleepy_X), 1)

                X_train.append((normal_X, sleepy_X))
                y_train.append((normal_y, sleepy_y))

                length_list.append((len(normal_y), len(sleepy_y)))

            X_test, y_test = [], []
            for subject in leave_out_subjects:
                print('test', subject)
                normal_frames = normal_frames_dict[str(subject)]
                normal_X = self.split_frames(normal_frames)
                normal_y = self.create_labels(len(normal_X), 0)

                sleepy_frames = sleepy_frames_dict[str(subject)]
                sleepy_X = self.split_frames(sleepy_frames)
                sleepy_y = self.create_labels(len(sleepy_X), 1)
                
                X_test.append((normal_X, sleepy_X))
                y_test.append((normal_y, sleepy_y))
                length_list.append((len(normal_y), len(sleepy_y)))

            return X_train, X_test, y_train, y_test, length_list
        
    def get_shallow_data(self, subjects, leave_out_subjects, segment_length, gru_segment_length):
            X_train, y_train = [], []
            length_list = []
            for sub in subjects:
                subject = 'subject' + str(sub)

                if sub in leave_out_subjects:
                    continue

                X_normal, X_sleepy = get_status_rates(subject, 10, segment_length)

                X_normal_segmented = split_into_segments(X_normal, gru_segment_length)
                X_sleepy_segmented = split_into_segments(X_sleepy, gru_segment_length)

                y_normal = self.create_labels(len(X_normal_segmented), 0)
                y_sleepy = self.create_labels(len(X_sleepy_segmented), 1)

                X_train.append((X_normal_segmented, X_sleepy_segmented))
                y_train.append((y_normal, y_sleepy))

                length_list.append((len(y_normal), len(y_sleepy)))
           
            X_test, y_test = [], []
            for sub in leave_out_subjects:
                subject = 'subject' + str(sub)
                print('test', subject)

                X_normal, X_sleepy = get_status_rates(subject, 10, segment_length)

                X_normal_segmented = split_into_segments(X_normal, gru_segment_length)
                X_sleepy_segmented = split_into_segments(X_sleepy, gru_segment_length)

                y_normal = self.create_labels(len(X_normal_segmented), 0)
                y_sleepy = self.create_labels(len(X_sleepy_segmented), 1)

                X_test.append((X_normal_segmented, X_sleepy_segmented))
                y_test.append((y_normal, y_sleepy))

                length_list.append((len(y_normal), len(y_sleepy)))

            return X_train, X_test, y_train, y_test, length_list

    def train(self, X_train_shallow, X_train_deep, y_train, num_epochs=10, batch_size=32, validation_split=0.2):
        self.batch_size = batch_size
        model = self.construct_model()
        early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

        model.fit([X_train_shallow, X_train_deep], y_train, epochs=num_epochs, batch_size=batch_size, validation_split=validation_split, callbacks=[early_stop])

        self.model = model
        print("Done with training")
        return model

    def test(self, X_test, y_test):
        model = self.model
        return model.evaluate(X_test, y_test, batch_size=self.batch_size)
    
    def get_f1(self, X_test, y_test):
        y_predict = model.predict(X_test, batch_size = self.batch_size)
        y_predict_int = np.round(y_predict).astype(int).flatten()
        f1 = f1_score(y_test_int, y_predict_int)
        return f1

In [5]:
# Hardcoded but works (BUT ONLY FOR MULTIPLE TEST SUBJECTS)
def equally_sized_lists(array1, array2):
    list1 = array1[0]
    list2 = array2[0]
    list3 = array1[1]
    list4 = array2[1]
    if len(list1) > len(list2):
        list1 = list1[:len(list2)]
    elif len(list1) < len(list2):
        list2 = list2[:len(list1)]
        
    if len(list3) > len(list4):
        list3 = list3[:len(list4)]
    elif len(list3) < len(list4):
        list4 = list4[:len(list3)]
    return [list1, list3], [list2, list4]

def create_equal_arrays(shallow, deep):
    first = True
    for sub in range(np.array(shallow).shape[0]):
        shallow_features = np.array(shallow[sub])
        deep_features = np.array(deep[sub])
        
        shallow_features, deep_features = equally_sized_lists(shallow_features, deep_features)
        if first:
            normal_shallow = shallow_features[0]
            sleepy_shallow = shallow_features[1]

            normal_deep = deep_features[0]
            sleepy_deep = deep_features[1]
            first = False
            continue
        normal_shallow = np.concatenate((normal_shallow, shallow_features[0]))
        sleepy_shallow = np.concatenate((sleepy_shallow, shallow_features[1]))
            
        normal_deep = np.concatenate((normal_deep, deep_features[0]))
        sleepy_deep = np.concatenate((sleepy_deep, deep_features[1]))
        
    # combine lists and shuffle
    shallow = np.concatenate((normal_shallow, sleepy_shallow))
    deep = np.concatenate((normal_deep, sleepy_deep))
            
    return shallow, deep

In [ ]:
detector = DrowsinessDetector(segment_length=60 * 46, gru_segment_length=6)

#subjects = [1,3,4,6,7,9,14,15,16,20,23,24]
subjects = [1,3,4,6,7,9,10,11,12,14,15,16,20,23,24,25]

leave_out_ratio = 0.3
#leave_out_subjects = np.random.choice(subjects, int(len(subjects) * leave_out_ratio), replace=False)
leave_out_subjects = [6,16,14]
print("Test subjects: ", leave_out_subjects)

normal_dict, sleepy_dict = load_multi_embeddings(subjects)
X_deep_train, X_deep_test, y_deep_train, y_deep_test, deep_len_list = detector.get_deep_data(normal_dict, sleepy_dict, subjects, leave_out_subjects)

segment_length = 10 * 46
gru_segment_length = 6
X_shallow_train, X_shallow_test, y_shallow_train, y_shallow_test, shallow_len_list = detector.get_shallow_data(subjects, leave_out_subjects, segment_length, gru_segment_length)

X_shallow_train, X_deep_train = create_equal_arrays(X_shallow_train, X_deep_train)
y_shallow_train, y_deep_train = create_equal_arrays(y_shallow_train, y_deep_train)

X_shallow_test, X_deep_test = create_equal_arrays(X_shallow_test, X_deep_test)
y_shallow_test, y_deep_test = create_equal_arrays(y_shallow_test, y_deep_test)

model = detector.train(X_shallow_train, X_deep_train, y_shallow_train, num_epochs=100, batch_size=8)
#detector.test(X_test, y_test)
#y_test, y_predict = detector.compare_predictions(X_test, y_test)
#print(y_test, y_predict)